In [1]:
print("PREPARANDO EL ENTORNO\n\n")
import os
 # Instalar SDK java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Descargar Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
# Descomprimir la versión de Spark
!tar xf spark-3.3.1-bin-hadoop3.tgz
# Establecer las variables de entorno
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"
# Descargar findspark
!pip install -q findspark
# Descargar pyspark
!pip install -q pyspark
print("\n\n******** INSTALACIÓN CORRECTA *******")

PREPARANDO EL ENTORNO


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.4 MB/s eta 0:00:00


******** INSTALACIÓN CORRECTA *******


In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
sc = spark.sparkContext

**Crear un Dataframe a partir de un RDD**

In [ ]:
# Creando el RDD
rdd = sc.parallelize([item for item in range(10)]).map(lambda x: (x, x**2))
rdd.collect()

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81)]

In [ ]:
# 1. Crear Dataframe sin Schema, se coloca el nombre de las columnas
df = rdd.toDF(['NUMERO', 'CUADRO'])

In [ ]:
# 1.1. Revisamos el esquema (nombre columna, tipo y si acepta nulos)
df.printSchema()

root
 |-- NUMERO: long (nullable = true)
 |-- CUADRO: long (nullable = true)



In [ ]:
# 1.2. Mostramos
df.show(5)

+------+------+
|NUMERO|CUADRO|
+------+------+
|     0|     0|
|     1|     1|
|     2|     4|
|     3|     9|
|     4|    16|
+------+------+
only showing top 5 rows



In [ ]:
# 2. Creamos Dataframe con esquema
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

rdd1 = sc.parallelize([(1, 'José', 35.5), (2, 'Teresa', 54.3), (3, 'Katia', 12.7)])

In [ ]:
# 2.1. Generamos 2 esquemas de diferentes formas
esquema1 = StructType(
    [
        StructField('ID', IntegerType(), True),
        StructField('NOMBRE', StringType(), True),
        StructField('SALDO', DoubleType(), True)
    ]
)

esquema2 = "`ID` INT, `NOMBRE` STRING, `SALDO` DOUBLE"

In [ ]:
# 2.2. Probamos el esquema1
df1 = spark.createDataFrame(rdd1, schema=esquema1)
df1.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- SALDO: double (nullable = true)



In [ ]:
df1.show(5)

+---+------+-----+
| ID|NOMBRE|SALDO|
+---+------+-----+
|  1|  José| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



In [ ]:
# 2.3. Probamos el esquema2
df2 = spark.createDataFrame(rdd1, schema=esquema2)
df2.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- NOMBRE: string (nullable = true)
 |-- SALDO: double (nullable = true)



In [ ]:
df2.show(5)

+---+------+-----+
| ID|NOMBRE|SALDO|
+---+------+-----+
|  1|  José| 35.5|
|  2|Teresa| 54.3|
|  3| Katia| 12.7|
+---+------+-----+



**Crear un DataFrame a partir de fuentes de datos**


In [ ]:
# Crear un dataframe desde un txt
df = spark.read.text("./data45/dataTXT.txt")

In [ ]:
df.show()

+--------------------+
|               value|
+--------------------+
|Estamos en el cur...|
|En este capítulo ...|
|En esta sección e...|
|y en este ejemplo...|
+--------------------+



In [ ]:
df.show(truncate=False)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|Estamos en el curso de pyspark                                         |
|En este capítulo estamos estudiando el API SQL de Saprk                |
|En esta sección estamos creado dataframes a partir de fuentes de datos,|
|y en este ejemplo creamos un dataframe a partir de un texto plano      |
+-----------------------------------------------------------------------+



In [ ]:
# Crear un dataframe desde un csv
df1 = spark.read.csv("./data45/dataCSV.csv")

In [ ]:
df1.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+--------------------+
|        _c0|          _c1|                 _c2|                 _c3|        _c4|                 _c5|                 _c6|    _c7|   _c8|     _c9|         _c10|                _c11|             _c12|            _c13|                _c14|                _c15|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|vid

In [ ]:
# indicamos la columna
df1 = spark.read.option('header', 'true').csv("./data45/dataCSV.csv")

In [ ]:
df1.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [ ]:
# con un delimitador diferente
df2 = spark.read.option('header', 'true').option('delimiter', '|').csv("./data45/dataTab.txt")

In [ ]:
df2.show()

+----+----+----------+-----+
|pais|edad|     fecha|color|
+----+----+----------+-----+
|  MX|  23|2021-02-21| rojo|
|  CA|  56|2021-06-10| azul|
|  US|  32|2020-06-02|verde|
+----+----+----------+-----+



In [ ]:
# Crear dataframe a través de un json usando schema
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

In [ ]:
json_schema = StructType(
    [
        StructField('color', StringType(), True),
        StructField('edad', IntegerType(), True),
        StructField('fecha', DateType(), True),
        StructField('pais', StringType(), True)
    ]
)

In [ ]:
df4 = spark.read.schema(json_schema).json("./data45/dataJSON.json")

In [ ]:
df4.show()

+-----+----+----------+----+
|color|edad|     fecha|pais|
+-----+----+----------+----+
| rojo|null|2021-02-21|  MX|
| azul|null|2021-06-10|  CA|
|verde|null|2020-06-02|  US|
+-----+----+----------+----+



In [ ]:
df4.printSchema()

root
 |-- color: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- fecha: date (nullable = true)
 |-- pais: string (nullable = true)



In [ ]:
# Crear un dataframe a partir de un archivo parquet
df5 = spark.read.parquet('./data45/dataPARQUET.parquet')

In [ ]:
df5.show()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13T17:13:...|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           Fal

In [ ]:
df6 = spark.read.format('parquet').load('./data45/dataPARQUET.parquet')

In [ ]:
df6.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
df6.show(truncate=False)

+-----------+-------------+--------------------------------------------------------------------------------------+---------------------+-----------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------

**Trabajar con columnas**

In [ ]:
df = spark.read.parquet('./data47/dataPARQUET.parquet')

In [ ]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
df.select('title').show(truncate = False)

+--------------------------------------------------------------------------------------+
|title                                                                                 |
+--------------------------------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                                                    |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)                        |
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons                                 |
|Nickelback Lyrics: Real or Fake?                                                      |
|I Dare You: GOING BALD!?                                                              |
|2 Weeks with iPhone X                                                                 |
|Roy Moore & Jeff Sessions Cold Open - SNL                                             |
|5 Ice Cream Gadgets put to the Test                                                   |
|The Greatest Showman

In [ ]:
# Otra forma de select
from pyspark.sql.functions import col

In [ ]:
df.select(col('title')).show(truncate = False)

+--------------------------------------------------------------------------------------+
|title                                                                                 |
+--------------------------------------------------------------------------------------+
|WE WANT TO TALK ABOUT OUR MARRIAGE                                                    |
|The Trump Presidency: Last Week Tonight with John Oliver (HBO)                        |
|Racist Superman | Rudy Mancuso, King Bach & Lele Pons                                 |
|Nickelback Lyrics: Real or Fake?                                                      |
|I Dare You: GOING BALD!?                                                              |
|2 Weeks with iPhone X                                                                 |
|Roy Moore & Jeff Sessions Cold Open - SNL                                             |
|5 Ice Cream Gadgets put to the Test                                                   |
|The Greatest Showman

**Transformaciones: funciones select y selectExpr**

In [ ]:
df = spark.read.parquet("./data48/datos.parquet")

In [ ]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col

In [ ]:
df.select(col('video_id')).show(truncate = False)

+-----------+
|video_id   |
+-----------+
|2kyS6SvSYSE|
|1ZAPwfrtAFY|
|5qpjK5DgCt4|
|puqaWrEC7tY|
|d380meD0W0M|
|gHZ1Qz0KiKM|
|39idVpFF7NQ|
|nc99ccSXST0|
|jr9QtXwC9vc|
|TUmyygCMMGA|
|9wRQljFNDW8|
|VifQlJit6A0|
|5E4ZBSInqUU|
|GgVmn66oK_A|
|TaTleo4cOs8|
|kgaO45SyaO4|
|ZAQs-ctOqXQ|
|YVfyYrEmzgM|
|eNSN6qet1kE|
|B5HORANmzHw|
+-----------+
only showing top 20 rows



In [ ]:
df.select('video_id', 'trending_date').show(truncate = False)

+-----------+-------------+
|video_id   |trending_date|
+-----------+-------------+
|2kyS6SvSYSE|17.14.11     |
|1ZAPwfrtAFY|17.14.11     |
|5qpjK5DgCt4|17.14.11     |
|puqaWrEC7tY|17.14.11     |
|d380meD0W0M|17.14.11     |
|gHZ1Qz0KiKM|17.14.11     |
|39idVpFF7NQ|17.14.11     |
|nc99ccSXST0|17.14.11     |
|jr9QtXwC9vc|17.14.11     |
|TUmyygCMMGA|17.14.11     |
|9wRQljFNDW8|17.14.11     |
|VifQlJit6A0|17.14.11     |
|5E4ZBSInqUU|17.14.11     |
|GgVmn66oK_A|17.14.11     |
|TaTleo4cOs8|17.14.11     |
|kgaO45SyaO4|17.14.11     |
|ZAQs-ctOqXQ|17.14.11     |
|YVfyYrEmzgM|17.14.11     |
|eNSN6qet1kE|17.14.11     |
|B5HORANmzHw|17.14.11     |
+-----------+-------------+
only showing top 20 rows



In [ ]:
# Se va a generar un error ya que se debe usar col
df.select(
    'likes',
    'dislikes',
    ('likes' - 'dislikes')
).show()

TypeError: ignored

In [ ]:
df.select(
    col('likes'),
    col('dislikes'),
    (col('likes') - col('dislikes')).alias('aceptación')
).show()

+------+--------+----------+
| likes|dislikes|aceptación|
+------+--------+----------+
| 57527|    2966|     54561|
| 97185|    6146|     91039|
|146033|    5339|    140694|
| 10172|     666|      9506|
|132235|    1989|    130246|
|  9763|     511|      9252|
| 15993|    2445|     13548|
| 23663|     778|     22885|
|  3543|     119|      3424|
| 12654|    1363|     11291|
|   655|      25|       630|
|  1576|     303|      1273|
|114188|    1333|    112855|
|  7848|    1171|      6677|
|  7473|     246|      7227|
|  9419|      52|      9367|
|  8011|     638|      7373|
|  5398|      53|      5345|
| 11963|      36|     11927|
|  8421|     191|      8230|
+------+--------+----------+
only showing top 20 rows



In [ ]:
# selectExpr

In [ ]:
df.selectExpr('likes', 'dislikes', '(likes - dislikes) as aceptacion').show()

+------+--------+----------+
| likes|dislikes|aceptacion|
+------+--------+----------+
| 57527|    2966|     54561|
| 97185|    6146|     91039|
|146033|    5339|    140694|
| 10172|     666|      9506|
|132235|    1989|    130246|
|  9763|     511|      9252|
| 15993|    2445|     13548|
| 23663|     778|     22885|
|  3543|     119|      3424|
| 12654|    1363|     11291|
|   655|      25|       630|
|  1576|     303|      1273|
|114188|    1333|    112855|
|  7848|    1171|      6677|
|  7473|     246|      7227|
|  9419|      52|      9367|
|  8011|     638|      7373|
|  5398|      53|      5345|
| 11963|      36|     11927|
|  8421|     191|      8230|
+------+--------+----------+
only showing top 20 rows



In [ ]:
df.selectExpr("count(distinct(video_id)) as videos").show()

+------+
|videos|
+------+
|  6837|
+------+



**Transformaciones: Funciones filter y where**

In [ ]:
from pyspark.sql.functions import col

In [ ]:
df.show()

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13 17:13:01|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           False| 

In [ ]:
# filter
df.filter(col('video_id') == '2kyS6SvSYSE').show()

+-----------+-------------+--------------------+-------------+-----------+-------------------+---------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|channel_title|category_id|       publish_time|           tags|  views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+-------------+-----------+-------------------+---------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...| CaseyNeistat|         22|2017-11-13 17:13:01|SHANtell martin| 748374|57527|    2966|        15954|https://i.ytimg.c...|            False|           False|                 False|SHANTELL'S CHANNE...|
|2kyS6Sv

In [ ]:
# where
df1 = spark.read.parquet('./data49/datos.parquet').where(col('trending_date') != '17.14.11').show()

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|            video_id|       trending_date|               title|       channel_title|         category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|\nCook with confi...|             recipes|              videos| and restaurant g...| dining destinations|               null|                

In [ ]:
df2 = spark.read.parquet('./data49/datos.parquet').where(col('likes') > 5000)

In [ ]:
df2.filter((col('trending_date') != '17.14.11') & (col('likes') > 7000)).show()

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|YvfYK0EEhK4|     17.15.11|Brent Pella - Why...|         Brent Pella|         23|2017-11-14 15:32:51|"spirit airlines"...| 462490| 14132|     795|          666|https://i.ytimg.c...|            False|           False| 

In [ ]:
df2.filter(col('trending_date') != '17.14.11').filter(col('likes') > 7000).show()

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|YvfYK0EEhK4|     17.15.11|Brent Pella - Why...|         Brent Pella|         23|2017-11-14 15:32:51|"spirit airlines"...| 462490| 14132|     795|          666|https://i.ytimg.c...|            False|           False| 

**Transformaciones: funciones distinct y dropDuplicates**

In [ ]:
df = spark.read.parquet('./data50')

In [ ]:
# distinct
df_sin_duplicados = df.distinct()

In [ ]:
print("El costeo del dataframe original es {}".format(df.count()))
print("El conteo del dataframe sin duplicados es {}".format(df_sin_duplicados.count()))

El costeo del dataframe original es 48137
El conteo del dataframe sin duplicados es 41428


In [ ]:
# dropDuplicates
dataframe = spark.createDataFrame([(1,'azul',567), (2,'rojo',487), (1,'azul',345), (2,'verde',783)]).toDF('id', 'color', 'importe')

In [ ]:
dataframe.show()

+---+-----+-------+
| id|color|importe|
+---+-----+-------+
|  1| azul|    567|
|  2| rojo|    487|
|  1| azul|    345|
|  2|verde|    783|
+---+-----+-------+



In [ ]:
dataframe.dropDuplicates(['id', 'color']).show()

+---+-----+-------+
| id|color|importe|
+---+-----+-------+
|  1| azul|    567|
|  2| rojo|    487|
|  2|verde|    783|
+---+-----+-------+



**Transformaciones: funciones sort y orderBy**

In [ ]:
from pyspark.sql.functions import col

df = (spark.read.parquet("./data51/")
      .select(col('likes'), col('views'), col('video_id'), col('dislikes'))
      .dropDuplicates(['video_id'])
)

In [ ]:
df.show()

+------+-------+--------------------+--------+
| likes|  views|            video_id|dislikes|
+------+-------+--------------------+--------+
| 63995|1525400|         bAkEd8r7Nnw|     896|
|   427|   9036|         eijd-yjXY9E|      14|
|  4145| 318249|         npcqBt_e4k0|     110|
|  6669| 203615|         LeWtF5y9-6Q|     136|
|  2166| 104499|         GhcqN2FDAnA|    1066|
| 10834| 160196|         v_CMMWCN5nQ|     162|
| 36068| 962042|         R8WBN3fJmwM|     845|
|   982|  36848|         oKuPJ7zF0_k|       6|
| 26482| 713615|         B3JFSL8AA70|    2443|
|275632|2822642|         f6Egj7ncOi8|    1444|
| 23922| 321885|         8gE6cek7F30|     317|
|    70|  13670|         EdkK29-TWJk|       1|
|  1131| 120802|         8szK9FBpdPI|      92|
| 12355| 294080|         6gFj1XJ6b5o|      80|
|  null|   null|\nhttp://www.Mast...|    null|
| 12070| 233766|         wOFuVNiAJQQ|     117|
| 21067| 210371|         PpElRBQ-yGc|     135|
|  4609| 363194|         q11UD-6XT-8|     955|
|   188|  311

In [ ]:
# sort
df.sort('likes').show()

+-----+-----+--------------------+--------+
|likes|views|            video_id|dislikes|
+-----+-----+--------------------+--------+
| null| null|\nFor more videos...|    null|
| null| null|\nFashion Editor:...|    null|
| null| null|\nAccess Hollywoo...|    null|
| null| null|\nStill haven’t s...|    null|
| null| null|\nhttps://www.you...|    null|
| null| null|Horror Outro ► ht...|    null|
| null| null|\nChapped lips ar...|    null|
| null| null|\nRoar: https://w...|    null|
| null| null|\nThe leading int...|    null|
| null| null|             \nToday|    null|
| null| null|\nONE STRANGE ROC...|    null|
| null| null|\nSNAPCHAT: fishi...|    null|
| null| null|\nInstagram: http...|    null|
| null| null|\nInstagram.com/w...|    null|
| null| null|\n5050 State Hwy....|    null|
| null| null|\nSIGN UP FOR BRA...|    null|
| null| null|\nJames Ambler an...|    null|
| null| null|\nhttp://www.Mast...|    null|
| null| null|\nEver After Tuto...|    null|
| null| null|          \nEvelin 

In [ ]:
from pyspark.sql.functions import  desc

In [ ]:
df.sort(desc('likes')).show()

+-------+--------+-----------+--------+
|  likes|   views|   video_id|dislikes|
+-------+--------+-----------+--------+
|3880071|39349927|7C2z4GqqS5E|   72707|
|2055137|13945717|kTlv5_Bs8aw|   23888|
|2050527|10695328|OK3GJ0WIQ8s|   14711|
|1956202|10666323|p8npDG2ulKQ|   13966|
|1735895|37736281|6ZfuNTqbHE8|   21969|
|1634124|33523622|2Vv-BfVoq4g|   21082|
|1572997| 7518332|kX0vO4vlJuU|    8113|
|1437859| 5884233|D_6QmL6rExk|    6390|
|1405355|31648454|VYOjWnS4cMY|   51547|
|1401915| 5275672|8O_MwlZ2dEg|    6268|
|1386616|15873034|ffxKSjUwKdU|   40714|
|1366736|16884972|J2HytHu5VBI|   59930|
|1290509| 6416697|2tDKp41nrw8|    4358|
|1207457|13754992|_5d-sQ7Fh5M|  280675|
|1167488| 8041970|oWjxSkJpxFU|  147643|
|1149185|24782158|FlsCjmMhFmw|  483924|
|1111592|38873543|i0p1bmr0EmE|   96407|
|1065777|14089954|dfnCAmr569k|   47839|
| 983693|14820746|tCXGJQYZ9JA|   44254|
| 975715|19716689|QwievZ1Tx-8|    9118|
+-------+--------+-----------+--------+
only showing top 20 rows



In [ ]:
# función orderBy
df.orderBy(col('views')).show()

+-----+-----+--------------------+--------+
|likes|views|            video_id|dislikes|
+-----+-----+--------------------+--------+
| null| null|\nIMDB - http://w...|    null|
| null| null|\nThis is the fir...|    null|
| null| null|\nAccess Hollywoo...|    null|
| null| null|\nStill haven’t s...|    null|
| null| null|\nhttps://www.you...|    null|
| null| null|          \nEvelin 7|    null|
| null| null|Horror Outro ► ht...|    null|
| null| null|\nChapped lips ar...|    null|
| null| null|\nRoar: https://w...|    null|
| null| null|\nThe leading int...|    null|
| null| null|             \nToday|    null|
| null| null|\nONE STRANGE ROC...|    null|
| null| null|\nSNAPCHAT: fishi...|    null|
| null| null|\nInstagram: http...|    null|
| null| null|\nInstagram.com/w...|    null|
| null| null|\n5050 State Hwy....|    null|
| null| null|\nFor more videos...|    null|
| null| null|\nJames Ambler an...|    null|
| null| null|\nFashion Editor:...|    null|
| null| null|\nEver After Tuto..

In [ ]:
df.orderBy(col('views').desc()).show()

+-------+--------+-----------+--------+
|  likes|   views|   video_id|dislikes|
+-------+--------+-----------+--------+
| 609101|48431654|-BQJo3vK8O8|   52259|
|3880071|39349927|7C2z4GqqS5E|   72707|
|1111592|38873543|i0p1bmr0EmE|   96407|
|1735895|37736281|6ZfuNTqbHE8|   21969|
|1634124|33523622|2Vv-BfVoq4g|   21082|
|1405355|31648454|VYOjWnS4cMY|   51547|
| 850362|27973210|u9Mv98Gr5pY|   26541|
|1149185|24782158|FlsCjmMhFmw|  483924|
| 641546|24421448|U9BwWKXjVaI|   16517|
| 587326|23758250|1J76wN0TPI4|   18799|
|      0|20921796|BhIEIO0vaBE|       0|
| 975715|19716689|QwievZ1Tx-8|    9118|
| 511753|18639195|rRr1qiJRsXk|   15606|
| 754791|18195959|rRzxEiBLQCA|   65326|
| 399200|18184886|vn9mMeWcgoM|   17473|
| 787419|17158531|n1WpP7iowLc|   43420|
|1366736|16884972|J2HytHu5VBI|   59930|
|1386616|15873034|ffxKSjUwKdU|   40714|
| 278743|15006579|yDiXQl7grPQ|   13599|
| 983693|14820746|tCXGJQYZ9JA|   44254|
+-------+--------+-----------+--------+
only showing top 20 rows



In [ ]:
dataframe = spark.createDataFrame([(1,'azul',568), (2,'rojo',235), (1,'azul',456), (2,'azul',783)]).toDF('id', 'color', 'importe')

In [ ]:
dataframe.show()

+---+-----+-------+
| id|color|importe|
+---+-----+-------+
|  1| azul|    568|
|  2| rojo|    235|
|  1| azul|    456|
|  2| azul|    783|
+---+-----+-------+



In [ ]:
dataframe.orderBy(col('color').desc(), col('importe')).show()

+---+-----+-------+
| id|color|importe|
+---+-----+-------+
|  2| rojo|    235|
|  1| azul|    456|
|  1| azul|    568|
|  2| azul|    783|
+---+-----+-------+



In [ ]:
# función limit
top_10 = df.orderBy(col('views').desc()).limit(10)

In [ ]:
top_10.show()

+-------+--------+-----------+--------+
|  likes|   views|   video_id|dislikes|
+-------+--------+-----------+--------+
| 609101|48431654|-BQJo3vK8O8|   52259|
|3880071|39349927|7C2z4GqqS5E|   72707|
|1111592|38873543|i0p1bmr0EmE|   96407|
|1735895|37736281|6ZfuNTqbHE8|   21969|
|1634124|33523622|2Vv-BfVoq4g|   21082|
|1405355|31648454|VYOjWnS4cMY|   51547|
| 850362|27973210|u9Mv98Gr5pY|   26541|
|1149185|24782158|FlsCjmMhFmw|  483924|
| 641546|24421448|U9BwWKXjVaI|   16517|
| 587326|23758250|1J76wN0TPI4|   18799|
+-------+--------+-----------+--------+



**Transformaciones: funciones withColumn y withColumnRenamed**

In [ ]:
df = spark.read.parquet("./data52/")

In [ ]:
# withColumn
from pyspark.sql.functions import col

df_valoracion = df.withColumn('valoración', col('likes') - col('dislikes'))

In [ ]:
df_valoracion.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)
 |-- valoración: integer (nullable = true)



In [ ]:
df_valoracion1 = (
    df.withColumn('valoracion', col('likes') - col('dislikes'))
    .withColumn('res_div', col('valoracion') % 10)
)

In [ ]:
df_valoracion1.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)
 |-- valoracion: integer (nullable = true)
 |-- res_div: integer (nullable = true)



In [ ]:
df_valoracion1.select(col('likes'), col('dislikes'), col('valoracion'), col('res_div')).show()

+------+--------+----------+-------+
| likes|dislikes|valoracion|res_div|
+------+--------+----------+-------+
| 57527|    2966|     54561|      1|
| 97185|    6146|     91039|      9|
|146033|    5339|    140694|      4|
| 10172|     666|      9506|      6|
|132235|    1989|    130246|      6|
|  9763|     511|      9252|      2|
| 15993|    2445|     13548|      8|
| 23663|     778|     22885|      5|
|  3543|     119|      3424|      4|
| 12654|    1363|     11291|      1|
|   655|      25|       630|      0|
|  1576|     303|      1273|      3|
|114188|    1333|    112855|      5|
|  7848|    1171|      6677|      7|
|  7473|     246|      7227|      7|
|  9419|      52|      9367|      7|
|  8011|     638|      7373|      3|
|  5398|      53|      5345|      5|
| 11963|      36|     11927|      7|
|  8421|     191|      8230|      0|
+------+--------+----------+-------+
only showing top 20 rows



In [ ]:
# withColumnRenamed
df_renombrado = df.withColumnRenamed('video_id', 'id')

In [ ]:
df_renombrado.printSchema()

root
 |-- id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
# Si no existe la columna no hace nada
df_error = df.withColumnRenamed('nombre_que_no_existe', 'otro_nombre')

**Transformaciones: Funciones drop, sample y randomSplit**

In [ ]:
df = spark.read.parquet("./data53/")

In [ ]:
# drop
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
df_util = df.drop("comments_disabled")
df_util.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
df_util = df.drop("comments_disabled", "ratings_disabled", "thumbnail_link")
df_util.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
# Si no existe la columna no arroja error
df_util = df.drop("comments_disabled", "ratings_disabled", "thumbnail_link", "cafe")
df_util.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [ ]:
# Sample
df_muestra = df.sample(0.8)

In [ ]:
num_filas = df.count()
num_filas_muestra = df_muestra.count()

print("El 80% de filas del dataframe original es {}".format(num_filas - (num_filas * 0.2)))
print("El número de filas del dataframe muestra es {}".format(num_filas_muestra))

El 80% de filas del dataframe original es 38509.6
El número de filas del dataframe muestra es 38461


In [ ]:
# con semilla aleatoria
df_muestra = df.sample(fraction=0.8, seed=1234)

In [ ]:
# con reemplazo
df_muestra = df.sample(withReplacement=True, fraction=0.8, seed=1234)

In [ ]:
# randomSplit
train, test = df.randomSplit([0.8, 0.2], seed=1234)

**Trabajar con datos incorrectos o faltantes**

In [ ]:
df = spark.read.parquet("./data54/")
df.count()

48137

In [ ]:
# forma 1
df.na.drop().count()

40379

In [ ]:
# forma 2
df.na.drop('any').count()

40379

In [ ]:
# forma 3
df.dropna().count()

40379

In [ ]:
# forma 4 - subset indica la columna a revisar
df.na.drop(subset=['views']).count()

40949

In [ ]:
# forma 5 - con 2 columnas
df.na.drop(subset=['views', 'dislikes']).count()

40949

In [ ]:
# forma 6
from pyspark.sql.functions import col

df.orderBy(col('views')).select(col('views'), col('likes'), col('dislikes')).show()

+-----+-----+--------+
|views|likes|dislikes|
+-----+-----+--------+
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
| null| null|    null|
+-----+-----+--------+
only showing top 20 rows



In [ ]:
# forma 7 - Llenamos los na con 0
df.fillna(0).orderBy(col('views')).select(col('views'), col('likes'), col('dislikes')).show()

+-----+-----+--------+
|views|likes|dislikes|
+-----+-----+--------+
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
|    0|    0|       0|
+-----+-----+--------+
only showing top 20 rows



In [ ]:
# forma 8 - llenamos con 0, pero solo las columnas nombradas en subset
df.fillna(0, subset=['likes', 'dislikes']).orderBy(col('views')).select(col('views'), col('likes'), col('dislikes')).show()

+-----+-----+--------+
|views|likes|dislikes|
+-----+-----+--------+
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
| null|    0|       0|
+-----+-----+--------+
only showing top 20 rows



**Acciones sobre un DataFrame en Spark SQL**

In [5]:
df = spark.read.parquet("./data55/")

In [6]:
# show()
df.show()

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13 17:13:01|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           False| 

In [7]:
df.show(5)

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...|        CaseyNeistat|         22|2017-11-13 17:13:01|     SHANtell martin| 748374| 57527|    2966|        15954|https://i.ytimg.c...|            False|           False| 

In [8]:
df.show(5, truncate=False)

+-----------+-------------+--------------------------------------------------------------+---------------------+-----------+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+------+--------+-------------+----------------------------------------------+-----------------+----------------+----------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [10]:
# take - genera una lista
df.take(1)

[Row(video_id='2kyS6SvSYSE', trending_date='17.14.11', title='WE WANT TO TALK ABOUT OUR MARRIAGE', channel_title='CaseyNeistat', category_id='22', publish_time=datetime.datetime(2017, 11, 13, 17, 13, 1), tags='SHANtell martin', views=748374, likes=57527, dislikes=2966, comment_count=15954, thumbnail_link='https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg', comments_disabled='False', ratings_disabled='False', video_error_or_removed='False', description="SHANTELL'S CHANNEL - https://www.youtube.com/shantellmartin\\nCANDICE - https://www.lovebilly.com\\n\\nfilmed this video in 4k on this -- http://amzn.to/2sTDnRZ\\nwith this lens -- http://amzn.to/2rUJOmD\\nbig drone - http://tinyurl.com/h4ft3oy\\nOTHER GEAR ---  http://amzn.to/2o3GLX5\\nSony CAMERA http://amzn.to/2nOBmnv\\nOLD CAMERA; http://amzn.to/2o2cQBT\\nMAIN LENS; http://amzn.to/2od5gBJ\\nBIG SONY CAMERA; http://amzn.to/2nrdJRO\\nBIG Canon CAMERA; http://tinyurl.com/jn4q4vz\\nBENDY TRIPOD THING; http://tinyurl.com/gw3ylz2\\nYOU NEED T

In [18]:
# head - muestra la cabecera - genera una lista
df.head(1)

[Row(video_id='2kyS6SvSYSE', trending_date='17.14.11', title='WE WANT TO TALK ABOUT OUR MARRIAGE', channel_title='CaseyNeistat', category_id='22', publish_time=datetime.datetime(2017, 11, 13, 17, 13, 1), tags='SHANtell martin', views=748374, likes=57527, dislikes=2966, comment_count=15954, thumbnail_link='https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg', comments_disabled='False', ratings_disabled='False', video_error_or_removed='False', description="SHANTELL'S CHANNEL - https://www.youtube.com/shantellmartin\\nCANDICE - https://www.lovebilly.com\\n\\nfilmed this video in 4k on this -- http://amzn.to/2sTDnRZ\\nwith this lens -- http://amzn.to/2rUJOmD\\nbig drone - http://tinyurl.com/h4ft3oy\\nOTHER GEAR ---  http://amzn.to/2o3GLX5\\nSony CAMERA http://amzn.to/2nOBmnv\\nOLD CAMERA; http://amzn.to/2o2cQBT\\nMAIN LENS; http://amzn.to/2od5gBJ\\nBIG SONY CAMERA; http://amzn.to/2nrdJRO\\nBIG Canon CAMERA; http://tinyurl.com/jn4q4vz\\nBENDY TRIPOD THING; http://tinyurl.com/gw3ylz2\\nYOU NEED T

In [19]:
# collect
df.select('likes').collect()

[Row(likes=57527),
 Row(likes=97185),
 Row(likes=146033),
 Row(likes=10172),
 Row(likes=132235),
 Row(likes=9763),
 Row(likes=15993),
 Row(likes=23663),
 Row(likes=3543),
 Row(likes=12654),
 Row(likes=655),
 Row(likes=1576),
 Row(likes=114188),
 Row(likes=7848),
 Row(likes=7473),
 Row(likes=9419),
 Row(likes=8011),
 Row(likes=5398),
 Row(likes=11963),
 Row(likes=8421),
 Row(likes=9586),
 Row(likes=3585),
 Row(likes=11758),
 Row(likes=1707),
 Row(likes=4884),
 Row(likes=8676),
 Row(likes=4687),
 Row(likes=9033),
 Row(likes=156),
 Row(likes=715),
 Row(likes=4035),
 Row(likes=119),
 Row(likes=787419),
 Row(likes=3781),
 Row(likes=1661),
 Row(likes=2486),
 Row(likes=7515),
 Row(likes=1318),
 Row(likes=38397),
 Row(likes=6927),
 Row(likes=5389),
 Row(likes=308),
 Row(likes=7),
 Row(likes=15186),
 Row(likes=4451),
 Row(likes=33505),
 Row(likes=3417),
 Row(likes=2017),
 Row(likes=35),
 Row(likes=45406),
 Row(likes=99086),
 Row(likes=205),
 Row(likes=15397),
 Row(likes=None),
 Row(likes=None),

**Escritura de DataFrame**

In [21]:
df = spark.read.parquet("./data56/")

In [22]:
df1 = df.repartition(2)

In [23]:
df1.write.format('csv').option('sep', '|').save('./output/csv')

In [24]:
# Reducimos a 1 particion y guadamos
df1.coalesce(1).write.format('csv').option('sep', '|').save('./output/csv1')

In [26]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [27]:
df.select('comments_disabled').distinct().show()

+-----------------+
|comments_disabled|
+-----------------+
|            False|
|             null|
| sports and more.|
|          Wiz Kid|
|             True|
|         farfalle|
+-----------------+



In [28]:
from pyspark.sql.functions import col

In [29]:
# filtramos o limpiamos
df_limpio = df.filter(col('comments_disabled').isin('True', 'False'))

In [30]:
# guardamos solo la columna nombrada
df_limpio.write.partitionBy('comments_disabled').parquet('./output/parquet')

**Persistencia de DataFrames**

In [31]:
df = spark.createDataFrame([(1,'a'), (2,'b'), (3,'c')], ['id', 'valor'])

In [32]:
df.show()

+---+-----+
| id|valor|
+---+-----+
|  1|    a|
|  2|    b|
|  3|    c|
+---+-----+



In [33]:
df.persist()

DataFrame[id: bigint, valor: string]

In [34]:
df.unpersist()

DataFrame[id: bigint, valor: string]

In [35]:
df.cache()

DataFrame[id: bigint, valor: string]

In [36]:
from pyspark.storagelevel import StorageLevel

In [37]:
df.persist(StorageLevel.DISK_ONLY)

DataFrame[id: bigint, valor: string]

In [38]:
df.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[id: bigint, valor: string]

**Ejercicios**

Los datos adjuntos a esta lección forman parte de la base de datos [NeurIPS 2020] Data Science for COVID-19 (DS4C) disponible en Kaggle. Estos datos hacen referencia a los casos de contagio de covid-19 en Corea del Sur.

El archivo csv Case contiene los casos reportados y el archivo csv PatientInfo contiene la información de los pacientes.

1. A partir del archivo csv Case, determine las tres ciudades con más casos confirmados de la enfermedad. La salida debe contener tres columnas: provincia, ciudad y casos confirmados. El resultado debe contener exactamente los tres nombre de ciudades con más casos confirmados ya que no se admiten otros valores.

In [88]:
df = spark.read.option('header', 'true').option('inferSchema', 'true').csv('./ejercicio/Case.csv')
df = df.withColumnRenamed(' case_id', 'case_id')
df.show(5)

+-------+--------+------------+-----+--------------------+---------+---------+----------+
|case_id|province|        city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|  Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
|1000002|   Seoul|   Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
|1000003|   Seoul|     Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
|1000004|   Seoul|Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
|1000005|   Seoul|   Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
+-------+--------+------------+-----+--------------------+---------+---------+----------+
only showing top 5 rows



In [89]:
from pyspark.sql.functions import desc, col
df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: boolean (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [93]:
df.filter((col('city') != '-') & (col('city') != 'from other city')).orderBy(desc('confirmed')).select('province', 'city', 'confirmed').show(3)

+--------+------------+---------+
|province|        city|confirmed|
+--------+------------+---------+
|   Daegu|      Nam-gu|     4511|
|   Daegu|Dalseong-gun|      196|
|   Seoul|  Yongsan-gu|      139|
+--------+------------+---------+
only showing top 3 rows



2. Cree un dataframe a partir del archivo csv PatientInfo. Asegúrese de que su dataframe no contenga pacientes duplicados.

In [94]:
df_patient = spark.read.option("header", "true").option('inferSchema', 'true').csv("./ejercicio/PatientInfo.csv")
df_patient.show(5)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|patient_id|   sex|age|country|province|       city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|      released_date|deceased_date|   state|
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+------------------+-------------------+-------------------+-------------+--------+
|1000000001|  male|50s|  Korea|   Seoul| Gangseo-gu|     overseas inflow|       null|            75|        2020-01-22|2020-01-23 00:00:00|2020-02-05 00:00:00|         null|released|
|1000000002|  male|30s|  Korea|   Seoul|Jungnang-gu|     overseas inflow|       null|            31|              null|2020-01-30 00:00:00|2020-03-02 00:00:00|         null|released|
|1000000003|  male|50s|  Korea|   Seoul|  Jongno-gu|contact with patient| 2002000001|

In [95]:
df_patient.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: string (nullable = true)
 |-- contact_number: string (nullable = true)
 |-- symptom_onset_date: string (nullable = true)
 |-- confirmed_date: timestamp (nullable = true)
 |-- released_date: timestamp (nullable = true)
 |-- deceased_date: timestamp (nullable = true)
 |-- state: string (nullable = true)



2.a. ¿Cuántos pacientes tienen informado por quién se contagiaron(columna infected_by)? Obtenga solo los pacientes que tengan informado por quién se contagiaron.

In [96]:
df_patient.select(col('patient_id')).count()

5165

In [97]:
df_patient.select(col('patient_id')).distinct().count()

5164

In [98]:
df_patient = df_patient.drop_duplicates(['patient_id'])

In [99]:
df_patient.count()

5164

In [100]:
from pyspark.sql.functions import count

In [102]:
df_patient.select(count('infected_by').alias('conteo')).show()

+------+
|conteo|
+------+
|  1346|
+------+



In [103]:
pacientes_info_contagios = df_patient.na.drop(subset=['infected_by'])

In [104]:
pacientes_info_contagios.count()

1346

2.b. A partir de la salida del inciso anterior obtenga solo los pacientes femeninos. La salida no debe contener las columnas released_date y deceased_date.

In [111]:
final_df = pacientes_info_contagios.filter((col('sex') == 'female') & (col('sex').isNotNull())).drop('released_date', 'deceased_date')
final_df.show()

+----------+------+---+-------+--------+-------------+--------------------+-----------+--------------+------------------+-------------------+--------+
|patient_id|   sex|age|country|province|         city|      infection_case|infected_by|contact_number|symptom_onset_date|     confirmed_date|   state|
+----------+------+---+-------+--------+-------------+--------------------+-----------+--------------+------------------+-------------------+--------+
|1000000005|female|20s|  Korea|   Seoul|  Seongbuk-gu|contact with patient| 1000000002|             2|              null|2020-01-31 00:00:00|released|
|1000000006|female|50s|  Korea|   Seoul|    Jongno-gu|contact with patient| 1000000003|            43|              null|2020-01-31 00:00:00|released|
|1000000010|female|60s|  Korea|   Seoul|  Seongbuk-gu|contact with patient| 1000000003|             6|              null|2020-02-05 00:00:00|released|
|1000000014|female|60s|  Korea|   Seoul|    Jongno-gu|contact with patient| 1000000013|       

2.c. Establezca el número de particiones del dataframe resultante del inciso anterior en dos. Escriba el dataframe resultante en un archivo parquet. La salida debe estar particionada por la provincia y el modo de escritura debe ser overwrite.

In [110]:
final_df.coalesce(2).write.partitionBy('province').mode('overwrite').parquet('./ejercicio/salida')